In [ ]:
import zipfile

def extract_zip_file(zip_file_path, extract_dir):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)

# 압축 파일 경로
zip_file_path = '/content/drive/MyDrive/pcbdata/pcb3.zip'

# 압축을 풀 폴더 경로
extract_dir = '/content/pcb3'

# 압축 풀기
extract_zip_file(zip_file_path, extract_dir)

In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense


In [ ]:
# 데이터셋 경로
train_dir = '/content/pcb3/traindata'
image_size = (512, 512)
batch_size = 50

In [ ]:
# ImageDataGenerator 생성
data_generator = ImageDataGenerator(rescale=1./255)



In [ ]:
# 훈련 데이터 로더 생성
train_generator = data_generator.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

Found 1150 images belonging to 5 classes.


In [ ]:
import tensorflow as tf
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(512, 512, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

In [ ]:
model.summary()

In [ ]:
# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])



In [ ]:
# 배치로 학습
steps_per_epoch = train_generator.n // batch_size
epochs = 10
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=epochs)

In [ ]:
from tensorflow import keras
#model.save('/content/drive/MyDrive/pcbdata/model/pcbblur10.h5')
#model = keras.models.load_model('/content/drive/MyDrive/pcbdata/model/pcbblur5.h5')

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score

# 테스트 데이터 준비
test_dir = '/content/pcb3/testdata'
image_size = (512, 512)
batch_size = 250

# ImageDataGenerator 생성 (전처리는 훈련 데이터와 동일하게)
test_data_generator = ImageDataGenerator(rescale=1./255)

# 테스트 데이터 로더 생성
test_generator = test_data_generator.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# 테스트 데이터로 예측 수행
y_true = test_generator.classes
y_pred_prob = model.predict(test_generator)
y_pred = np.argmax(y_pred_prob, axis=1)

# 정확도 평가
accuracy = accuracy_score(y_true, y_pred)
print('테스트 데이터 정확도:', accuracy)

In [ ]:
import sklearn
import itertools
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    plt.figure(figsize = (5,5))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

    plt.show()

In [ ]:
class1 = ['o','g','m','w','r']
#Y_pred_classes = np.argmax(Y_pred,axis=1)
confusion_mtx = confusion_matrix(y_true, y_pred)
plot_confusion_matrix(confusion_mtx, classes = list(class1))

In [ ]:
import cv2
import numpy as np
lapp = []
def show_images_from_batches(test_generator, num_batches=1):
    nn=0
    for _ in range(num_batches):
        # 제네레이터에서 배치 하나 가져오기
        batch = next(test_generator)

        # 배치에서 이미지 하나씩 출력
        for image in batch[0]:
            nn=nn+1
            # 이미지 출력 또는 원하는 작업 수행
            image_uint8 = (image * 255).astype(np.uint8)

            # 그레이스케일 이미지로 변환
            gray = cv2.cvtColor(image_uint8, cv2.COLOR_BGR2GRAY)
            src = cv2.resize(gray, (256, 256))
            gray = src


            # 라플라시안 필터 적용
            laplacian = cv2.Laplacian(gray, cv2.CV_8U, ksize=3)

            # 라플라시안 필터 적용된 이미지에서, 각 값들의 제곱을 구함
            dst = cv2.sqrBoxFilter(laplacian, ddepth=-1, ksize=(3,3), normalize=True)

            # 분산을 구함 = 제곱의 평균 - 평균의 제곱
            z = (dst - (laplacian*laplacian))

            # blurred 처리되었다고 판단할 기준값 설정

            threshold = 200

            z[z<threshold] = 0
            z[z>=threshold] = 1





            # counting 0 and 1 (0 is blurred, 1 is ok
            blurred = z[z==0].shape[0]
            normal = z[z==1].shape[0]

            # blurred 처리된 영역의 비율 계산
            confidence = normal / (blurred+normal)

            lapp.append(confidence)

show_images_from_batches(test_generator, num_batches=1)

In [ ]:
#y_pred = np.argmax(y_pred_prob, axis=1)
for i in range(0,250):
  if lapp[i]>0.78:
    y_pred_prob[0] = 1
  if lapp[i]<0.36:
    y_pred_prob[0] = 0

In [ ]:
y_pred = np.argmax(y_pred_prob, axis=1)

class1 = ['o','g','m','w','r']
#Y_pred_classes = np.argmax(Y_pred,axis=1)
confusion_mtx = confusion_matrix(y_true, y_pred)
plot_confusion_matrix(confusion_mtx, classes = list(class1))